# Imports

In [8]:
import requests
import os

## Get API key

Make sure to set the API key as a environment variable in your system:

For Ubuntu:
- `export TICKETMASTER_API_KEY="LXVLySLQxPIExSYz"`

With Python:
- `os.environ["TICKETMASTER_API_KEY"] = "YOUR_API_KEY"`

In [9]:
os.environ["TICKETMASTER_API_KEY"] = "xkwIHOr5jYGxFVPOCeIF9JMeP2YPjQzH"

In [10]:
API_KEY = os.environ.get("TICKETMASTER_API_KEY")
BASE_URL = "https://app.ticketmaster.com/discovery/v2"

if API_KEY is None:
    raise ValueError("Please set the TICKETMASTER_API_KEY environment variable.")

In [11]:
API_KEY

'xkwIHOr5jYGxFVPOCeIF9JMeP2YPjQzH'

## Get events with requests package

In [9]:
url = f"{BASE_URL}/events.json"

# Set the parameters for the request
params = {
    "size": 5,
    "countryCode": "NL",
    "city": "Eindhoven",
    "radius": 50,
    "unit": "km",
    "apikey": API_KEY
}

# Make the GET request to the Ticketmaster API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse and print the JSON response
    data = response.json()
    print(data)
else:
    print(f"Failed to retrieve data: {response.status_code}")

{'_embedded': {'events': [{'name': 'Allman Brown', 'type': 'event', 'id': 'Z698xZbpZ17Ga0g', 'test': False, 'url': 'https://www.ticketmaster.nl/event/allman-brown-tickets/309935?language=en-us', 'locale': 'en-us', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_SOURCE', 'width': 1440, 'height': 810, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_TABLET_LANDSCAPE_LARGE_16_9.jpg', 'width': 2048, 'height': 1152, 'fallback': True}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_RETINA_PORTRAIT_3_2.jpg', 'width': 640, 'height': 427, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_ARTIST_PAGE_3_2.jpg', 'width': 305, 'height': 203, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_TA

In [10]:
data["page"]

{'size': 5, 'totalElements': 18, 'totalPages': 4, 'number': 0}

In [11]:
data["_embedded"]["events"][1]

{'name': 'James Leg',
 'type': 'event',
 'id': 'Z698xZbpZ17GF-t',
 'test': False,
 'url': 'https://www.ticketmaster.nl/event/james-leg-tickets/308869?language=en-us',
 'locale': 'en-us',
 'images': [{'ratio': '4_3',
   'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_CUSTOM.jpg',
   'width': 305,
   'height': 225,
   'fallback': False},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_RECOMENDATION_16_9.jpg',
   'width': 100,
   'height': 56,
   'fallback': False},
  {'ratio': '1_1',
   'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_SOURCE',
   'width': 720,
   'height': 720,
   'fallback': False},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_ARTIST_PAGE_3_2.jpg',
   'width': 305,
   'height': 203,
   'fallback': False},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/c/fbc/b293c0ad-c904-4215-bc59-8d7f2414dfbc_10

## Get events locally

In [ ]:
import requests
import os
import json
import datetime

BASE_API_URL = "https://app.ticketmaster.com/discovery/v2"
EVENTS_ENDPOINT = f"{BASE_API_URL}/events.json"
MAX_PAGES = 1
PAGE_SIZE = 20


def paginated_getter(api_key: str, events_until: datetime.date, country_code: str = "NL"):
    """Generator function to retrieve paginated events from Ticketmaster API."""
    page_number = 1

    while page_number <= MAX_PAGES:
        params = {
            "page": page_number,
            "size": PAGE_SIZE,
            "countryCode": country_code,
            "endDateTime": events_until.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "apikey": api_key
        }

        try:
            response = requests.get(EVENTS_ENDPOINT, params=params)
            response.raise_for_status()
            page_json = response.json()
            events = page_json.get("_embedded", {}).get("events", [])

            if not events:
                print(f'No more events available after page {page_number}.')
                break

            yield page_json
            print(f'Retrieved page {page_number} with {len(events)} event(s).')

            page_number += 1

        except requests.exceptions.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            break
        except requests.exceptions.RequestException as e:
            print(f"Request exception occurred: {e}")
            break


def save_events_to_file(data, directory, filename):
    """Save the event data to a JSON file."""

    file_path = os.path.join(directory, filename)
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    print(f"Data successfully written to {file_path}")
    
    return file_path


import sys
import urllib.request
from concurrent.futures import ThreadPoolExecutor
from google.cloud import storage
from google.api_core.exceptions import NotFound, Forbidden
import time

# Change this to your bucket name
BUCKET_NAME = "ticketmaster_bucket"

# If you authenticated through the GCP SDK you can comment out these two lines
CREDENTIALS_FILE = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
client = storage.Client.from_service_account_json(CREDENTIALS_FILE)

bucket = client.bucket(BUCKET_NAME)

def verify_gcs_upload(blob_name):
    return storage.Blob(bucket=bucket, name=blob_name).exists(client)

def upload_to_gcs(file_path, max_retries=3):
    blob_name = os.path.basename(file_path)
    blob = bucket.blob(blob_name)

    for attempt in range(max_retries):
        try:
            print(f"Uploading {file_path} to {BUCKET_NAME} (Attempt {attempt + 1})...")
            blob.upload_from_filename(file_path)
            print(f"Uploaded: gs://{BUCKET_NAME}/{blob_name}")

            if verify_gcs_upload(blob_name):
                print(f"Verification successful for {blob_name}")
                return
            else:
                print(f"Verification failed for {blob_name}, retrying...")

        except Exception as e:
            print(f"Failed to upload {file_path} to GCS: {e}")

        time.sleep(5)

    print(f"Giving up on {file_path} after {max_retries} attempts.")


def main():
    api_key = os.environ.get("TICKETMASTER_API_KEY")
    country_code = "NL"
    data_dir = "data"

    cities = ["Eindhoven", "Amsterdam"]
    today = datetime.date.today()
    events_until = today + datetime.timedelta(weeks=4)

    datetime_now = datetime.datetime.now().strftime("%Y%m%d%Hh")

    os.makedirs(data_dir, exist_ok=True)

    local_file_paths = []
    for city in cities:
        print(f"Fetching events for {city}...")
        for response in paginated_getter(api_key, city, events_until, country_code):
            page_number = response["page"]["number"]
            file_name = f"{datetime_now}_events_pag{str(page_number).zfill(3)}_{country_code}_{city.lower()}.json"
            local_file_path = save_events_to_file(response, data_dir, file_name)
            local_file_paths.append(local_file_path)

    for file_path in local_file_paths:
        upload_to_gcs(file_path)



if __name__ == '__main__':
    main()

Fetching events for Eindhoven...
Data successfully written to data/2025041113h_events_pag001_NL_eindhoven.json
Retrieved page 1 with 20 event(s).
Fetching events for Amsterdam...
Data successfully written to data/2025041113h_events_pag001_NL_amsterdam.json
Retrieved page 1 with 20 event(s).
Uploading data/2025041113h_events_pag001_NL_eindhoven.json to ticketmaster_bucket (Attempt 1)...
Uploaded: gs://ticketmaster_bucket/2025041113h_events_pag001_NL_eindhoven.json
Verification successful for 2025041113h_events_pag001_NL_eindhoven.json
Uploading data/2025041113h_events_pag001_NL_amsterdam.json to ticketmaster_bucket (Attempt 1)...
Uploaded: gs://ticketmaster_bucket/2025041113h_events_pag001_NL_amsterdam.json
Verification successful for 2025041113h_events_pag001_NL_amsterdam.json


In [55]:
local_file_paths

NameError: name 'local_file_paths' is not defined

In [ ]:
import requests
import pandas as pd
import dlt
import os
import json
import datetime

BASE_API_URL = "https://app.ticketmaster.com/discovery/v2"
EVENTS_ENDPOINT = f"{BASE_API_URL}/events.json"

def paginated_getter(API_KEY: str, city: str, events_until: datetime.date, country_code: str = "NL"):
    """
    Generator function to retrieve paginated events from Ticketmaster API.
    """

    page_number = 1
    MAX_PAGES = 1
    PAGE_SIZE = 2

    while page_number <= MAX_PAGES:
        params = {
            "page": page_number,
            # "size": PAGE_SIZE,
            "countryCode": country_code,
            "city": city,
            "endDateTime": events_until.strftime("%Y-%m-%dT%H:%M:%SZ"), # Filter events with a start date before until date
            "apikey": API_KEY
        }

        try:
            response = requests.get(EVENTS_ENDPOINT, params=params)
            response.raise_for_status()
            page_json = response.json()
            events = page_json.get("_embedded", {}).get("events", [])

            if not page_json:
                print(f'No more events available after page {page_number}.')
                break
            
            yield page_json
            print(f'Retrieved page {page_number} with {len(events)} event(s).')

            page_number += 1
            
        except requests.exceptions.HTTPError as e:
            # raise(f"HTTP error occurred: {e}")
            print(f"HTTP error occurred: {e}")
            break
        except requests.exceptions.RequestException as e:
            print(f"Request exception occurred: {e}")
            break

if __name__ == '__main__':

    API_KEY = os.environ.get("TICKETMASTER_API_KEY")
    COUNTRY_CODE = "NL"
    DATA_DIR = "data"

    cities = ["Eindhoven", "Amsterdam"]
    today = datetime.date.today()
    events_until = today + datetime.timedelta(weeks=4)

    datetime_now = datetime.datetime.now().strftime("%Y%m%d%HH")

    for city in cities:
        for response in paginated_getter(
            API_KEY,
            city=city,
            events_until=events_until,
            country_code=COUNTRY_CODE
        ):

            page_number = response["page"]["number"]

            file_name = f"{datetime_now}_events_pag{str(page_number).zfill(3)}_{COUNTRY_CODE}_{city}.json"
            file_name = file_name.lower()

            with open(os.path.join(DATA_DIR, file_name), 'w') as json_file:
                json.dump(response, json_file, indent=4)

Retrieved page 1 with 0 event(s).
Retrieved page 1 with 20 event(s).


In [28]:
import json

with open("events.json", 'w') as json_file:
    json.dump(events, json_file, indent=4)

print(f"Data successfully written")

Data successfully written


In [ ]:
generators_pipeline = dlt.pipeline(
    destination='duckdb', 
    dataset_name='generators'
)


# we can load any generator to a table at the pipeline destination as follows:
generators_pipeline.run(paginated_getter(),
            table_name="events",
            write_disposition="replace")

In [ ]:
import pandas as pd

df = pd.json_normalize(events)

,name,type,id,test,url,locale,images,classifications,promoters,sales.public.startDateTime,...,ticketing.safeTix.enabled,ticketing.id,_links.self.href,_links.attractions,_links.venues,_embedded.venues,_embedded.attractions,dates.access.startDateTime,dates.access.startApproximate,dates.access.endApproximate
0,Temple Fang + Mojo & The Kitchen Brothers,event,Z698xZbpZ1kFkZ-7a,False,https://www.ticketmaster.nl/event/temple-fang-...,en-us,"[{'ratio': '16_9', 'url': 'https://s1.ticketm....","[{'primary': True, 'segment': {'id': 'KZFzniwn...","[{'id': '3049', 'name': 'De Effenaar'}]",2024-11-21T09:00:00Z,...,False,ticketing,/discovery/v2/events/Z698xZbpZ1kFkZ-7a?locale=...,[{'href': '/discovery/v2/attractions/K8vZ9179B...,[{'href': '/discovery/v2/venues/Z598xZbpZd6v7?...,"[{'name': 'Effenaar', 'type': 'venue', 'id': '...","[{'name': 'Temple Fang', 'type': 'attraction',...",NaN,NaN,NaN
1,STRIPE-FEST,event,Z698xZbpZ16v77_4aA,False,https://www.ticketmaster.nl/event/stripe-fest-...,en-us,"[{'ratio': '4_3', 'url': 'https://s1.ticketm.n...","[{'primary': False, 'segment': {'id': 'KZFzniw...","[{'id': '5609', 'name': 'Fiesta BV'}]",2024-12-20T09:00:00Z,...,False,ticketing,/discovery/v2/events/Z698xZbpZ16v77_4aA?locale...,[{'href': '/discovery/v2/attractions/K8vZ917qP...,[{'href': '/discovery/v2/venues/Z598xZbpZkkae?...,"[{'name': 'Ketelhuisplein / Strijp-S', 'type':...","[{'name': 'STRIPE-FEST', 'type': 'attraction',...",2025-05-24T12:30:00Z,False,False


In [16]:
import requests

BASE_API_URL = "https://app.ticketmaster.com/discovery/v2"
URL = f"{BASE_URL}/events.json"
MAX_PAGES = 2

def paginated_getter():
    page_number = 1

    while True:

        # Set the parameters for the request
        params = {
            "page": page_number,
            "size": 1,
            "countryCode": "NL",
            "city": "Eindhoven",
            "radius": 50,
            "unit": "km",
            "apikey": API_KEY
        }

        # Make the GET request to the API
        response = requests.get(URL, params=params)
        response.raise_for_status()  # Raise an HTTP Error for bad responses
        page_json = response.json()
        page_data = page_json.get("_embedded", {}).get("events", [])

        # If the page has no records, stop iterating
        if (len(page_data) > 0) & (page_number <= MAX_PAGES):
            yield page_data
            print(f'got page number {page_number} with {len(page_data)} records')
            page_number += 1
        else:
            # No more data, break the loop
            break

if __name__ == '__main__':
    # Use the generator to iterate over pages
    for page_data in paginated_getter():
        # Process each page as needed
        print("page_data", page_data)


page_data [{'name': 'James Leg', 'type': 'event', 'id': 'Z698xZbpZ17GF-t', 'test': False, 'url': 'https://www.ticketmaster.nl/event/james-leg-tickets/308869?language=en-us', 'locale': 'en-us', 'images': [{'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_CUSTOM.jpg', 'width': 305, 'height': 225, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_RECOMENDATION_16_9.jpg', 'width': 100, 'height': 56, 'fallback': False}, {'ratio': '1_1', 'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_SOURCE', 'width': 720, 'height': 720, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/93a/312622a1-4c6a-4ace-8fbe-b7670aa2493a_ARTIST_PAGE_3_2.jpg', 'width': 305, 'height': 203, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/c/fbc/b293c0ad-c904-4215-bc59-8d7f2414dfbc_106141_TABLET_LANDSCAPE_3_2.jpg', 'width': 1024, 'height

In [21]:
import os
import sys
import urllib.request
from concurrent.futures import ThreadPoolExecutor
from google.cloud import storage
from google.api_core.exceptions import NotFound, Forbidden
import time

# Change this to your bucket name
BUCKET_NAME = "ticketmaster_bucket"

# If you authenticated through the GCP SDK you can comment out these two lines
CREDENTIALS_FILE = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
client = storage.Client.from_service_account_json(CREDENTIALS_FILE)

BASE_API_URL = "https://app.ticketmaster.com/discovery/v2"
EVENTS_ENDPOINT = f"{BASE_API_URL}/events.json"
CITIES = ["Eindhoven", "Utrecht", "Amsterdam"]

bucket = client.bucket(BUCKET_NAME)


MAX_PAGES = 2
PAGE_SIZE = 1
COUNTRY_CODE = "NL"


In [ ]:
import os
import sys
import urllib.request
from concurrent.futures import ThreadPoolExecutor
from google.cloud import storage
from google.api_core.exceptions import NotFound, Forbidden
import time

# Change this to your bucket name
BUCKET_NAME = "ticketmaster_bucket"

# If you authenticated through the GCP SDK you can comment out these two lines
CREDENTIALS_FILE = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
client = storage.Client.from_service_account_json(CREDENTIALS_FILE)

bucket = client.bucket(BUCKET_NAME)

def create_bucket(bucket_name):
    try:
        # Get bucket details
        bucket = client.get_bucket(bucket_name)

        # Check if the bucket belongs to the current project
        project_bucket_ids = [bckt.id for bckt in client.list_buckets()]
        if bucket_name in project_bucket_ids:
            print(
                f"Bucket '{bucket_name}' exists and belongs to your project. Proceeding..."
            )
        else:
            print(
                f"A bucket with the name '{bucket_name}' already exists, but it does not belong to your project."
            )
            sys.exit(1)

    except NotFound:
        # If the bucket doesn't exist, create it
        bucket = client.create_bucket(bucket_name)
        print(f"Created bucket '{bucket_name}'")
    except Forbidden:
        # If the request is forbidden, it means the bucket exists but you don't have access to see details
        print(
            f"A bucket with the name '{bucket_name}' exists, but it is not accessible. Bucket name is taken. Please try a different bucket name."
        )
        sys.exit(1)

def verify_gcs_upload(blob_name):
    return storage.Blob(bucket=bucket, name=blob_name).exists(client)

def upload_to_gcs(file_path, max_retries=3):
    blob_name = os.path.basename(file_path)
    blob = bucket.blob(blob_name)

    create_bucket(BUCKET_NAME)

    for attempt in range(max_retries):
        try:
            print(f"Uploading {file_path} to {BUCKET_NAME} (Attempt {attempt + 1})...")
            blob.upload_from_filename(file_path)
            print(f"Uploaded: gs://{BUCKET_NAME}/{blob_name}")

            if verify_gcs_upload(blob_name):
                print(f"Verification successful for {blob_name}")
                return
            else:
                print(f"Verification failed for {blob_name}, retrying...")
        except Exception as e:
            print(f"Failed to upload {file_path} to GCS: {e}")

        time.sleep(5)

    print(f"Giving up on {file_path} after {max_retries} attempts.")

In [23]:
create_bucket(BUCKET_NAME)

Bucket 'ticketmaster_bucket' exists and belongs to your project. Proceeding...


In [ ]:
verify_gcs_upload

In [ ]:
@dlt.source(name="rides")
def download_parquet():
     for month in range(1,7):
      file_name = f"yellow_tripdata_2024-0{month}.parquet"

      url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-0{month}.parquet"
      response = requests.get(url)

      df = pd.read_parquet(BytesIO(response.content))

      # Return the dataframe as a dlt resource for ingestion
      yield dlt.resource(df, name=file_name)

In [ ]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(
    destination='duckdb', 
    dataset_name='generators'
)


# we can load any generator to a table at the pipeline destination as follows:
generators_pipeline.run(paginated_getter(),
            table_name="events",
            write_disposition="replace")

got page number 1 with 1 records
got page number 2 with 1 records


LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7fd1413686e0>, metrics={'1744357845.2857563': [{'started_at': DateTime(2025, 4, 11, 7, 50, 46, 444082, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2025, 4, 11, 7, 50, 47, 986582, tzinfo=Timezone('UTC')), 'job_metrics': {'events___embedded__attractions__external_links__homepage.8d95b147a7.insert_values': LoadJobMetrics(job_id='events___embedded__attractions__external_links__homepage.8d95b147a7.insert_values', file_path='/home/lars/.dlt/pipelines/dlt_ipykernel_launcher/load/normalized/1744357845.2857563/started_jobs/events___embedded__attractions__external_links__homepage.8d95b147a7.0.insert_values', table_name='events___embedded__attractions__external_links__homepage', started_at=DateTime(2025, 4, 11, 7, 50, 46, 740577, tzinfo=Timezone('UTC')), finished_at=DateTime(2025, 4, 11, 7, 50, 46, 770689, tzinfo=Timezone('UTC')), state='completed', remote_url=None), 'events___embedded__attractions__external_links__musicbrainz.e2a

In [19]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


Loaded tables: 


┌────────────────────────────────────────────────────────────────┐
│                              name                              │
│                            varchar                             │
├────────────────────────────────────────────────────────────────┤
│ _dlt_loads                                                     │
│ _dlt_pipeline_state                                            │
│ _dlt_version                                                   │
│ events                                                         │
│ events___embedded__attractions                                 │
│ events___embedded__attractions__classifications                │
│ events___embedded__attractions__external_links__facebook       │
│ events___embedded__attractions__external_links__homepage       │
│ events___embedded__attractions__external_links__instagram      │
│ events___embedded__attractions__external_links__itunes         │
│                           ·                                 

In [21]:
events = conn.sql("SELECT * FROM events___embedded__venues").df()
display(events)

,name,type,id,test,url,locale,postal_code,timezone,city__name,country__name,...,location__longitude,location__latitude,upcoming_events__mfx_nl,upcoming_events___total,upcoming_events___filtered,_links__self__href,_dlt_parent_id,_dlt_list_idx,_dlt_id,upcoming_events__universe
0,Cafe Wilhelmina,venue,Z598xZbpZ77v7,False,https://www.ticketmaster.nl/venue/cafe-wilhelm...,en-us,5611 HE,Europe/Amsterdam,Eindhoven,Netherlands,...,5.4718,51.43788,4,4,0,/discovery/v2/venues/Z598xZbpZ77v7?locale=en-us,wR5G4Jf+6NPmfg,0,35gKxNeOnSiOOg,<NA>
1,Effenaar,venue,Z598xZbpZd6v7,False,https://www.ticketmaster.nl/venue/effenaar-ein...,en-us,5611 CK,Europe/Amsterdam,Eindhoven,Netherlands,...,5.48362,51.44158,74,77,0,/discovery/v2/venues/Z598xZbpZd6v7?locale=en-us,pHtuvPyux73eiw,0,uR+sFd2m40nVkQ,3


In [ ]:
conn.sql("SELECT * FROM events").df()

In [12]:
import dlt
from dlt.sources.rest_api import rest_api_source

url = f"{BASE_URL}/events.json"

# Set the parameters for the request
params = {
    "size": 5,
    "countryCode": "NL",
    "city": "Eindhoven",
    "radius": 50,
    "unit": "km",
    "apikey": API_KEY
}

# Configure the REST API source for Ticketmaster
ticketmaster_source = rest_api_source({
    "client": {
        "base_url": BASE_URL,
        "auth": {
            "token": API_KEY,
        },
        "paginator": {
            "type": "json_link",
            "next_url_path": "_links.next",
        },
    },
    "resources": [
        # "events" will be used as the endpoint path, the resource name,
        # and the table name in the destination.
        "events",
        {
            "name": "event_details",
            "endpoint": {
                "path": "/events.json",
                "params": {
                    "size": 5,
                    "countryCode": "NL",
                    "city": "Eindhoven",
                    "radius": 50,
                    "unit": "km",
                    "apikey": API_KEY
                }
            },
        }
    ],
})

# Initialize the pipeline
pipeline = dlt.pipeline(
    pipeline_name="ticketmaster_rest_api",
    destination="duckdb",
    dataset_name="ticketmaster_data",
)

# Run the pipeline to load data
load_info = pipeline.run(ticketmaster_source)

PipelineStepFailed: Pipeline execution failed at stage extract when processing package 1744314775.9828188 with exception:

<class 'dlt.extract.exceptions.ResourceExtractionError'>
In processing pipe events: extraction of resource events in generator paginate_resource caused an exception: 401 Client Error: Unauthorized for url: https://app.ticketmaster.com/discovery/v2/events

In [8]:
import dlt

# Initialize a DLT pipeline
pipeline = dlt.pipeline(
    pipeline_name='ticketmaster_events', 
    destination='duckdb', 
    dataset_name='events_data'
)

url = f"{BASE_URL}/events.json"
params = {
    "size": 5,
    "countryCode": "NL",
    "city": "Eindhoven",
    "radius": 50,
    "unit": "km",
    "apikey": API_KEY
}

# Fetch data using DLT's request
@dlt.source(name='ticketmaster_source')
def fetch_ticketmaster_events():
    # Perform a GET request to the API
    response = pipeline.request.get(url, params=params)
    response.raise_for_status()  # Raise an exception for HTTP error codes

    # Return the JSON data
    return response.json()

# Run the pipeline to load data
source = fetch_ticketmaster_events()
pipeline.run(source)

AttributeError: 'Pipeline' object has no attribute 'request'

In [35]:
event_id = "Z698xZbpZ17Ga0g"
url = f"{BASE_URL}/events/{event_id}"

# Set the parameters for the request
params = {
    "apikey": API_KEY
}

# Make the GET request to the Ticketmaster API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse and print the JSON response
    data = response.json()
    print(data)
else:
    print(f"Failed to retrieve data: {response.status_code}")

{'name': 'Allman Brown', 'type': 'event', 'id': 'Z698xZbpZ17Ga0g', 'test': False, 'url': 'https://www.ticketmaster.nl/event/allman-brown-tickets/309935', 'locale': 'en-us', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_SOURCE', 'width': 1440, 'height': 810, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_TABLET_LANDSCAPE_LARGE_16_9.jpg', 'width': 2048, 'height': 1152, 'fallback': True}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_RETINA_PORTRAIT_3_2.jpg', 'width': 640, 'height': 427, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_ARTIST_PAGE_3_2.jpg', 'width': 305, 'height': 203, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_TABLET_LANDSCAPE_3_2.jpg', 'width': 1024, '

In [36]:
response.url

'https://app.ticketmaster.com/discovery/v2/events/Z698xZbpZ17Ga0g?apikey=xkwIHOr5jYGxFVPOCeIF9JMeP2YPjQzH'

In [37]:
data

{'name': 'Allman Brown',
 'type': 'event',
 'id': 'Z698xZbpZ17Ga0g',
 'test': False,
 'url': 'https://www.ticketmaster.nl/event/allman-brown-tickets/309935',
 'locale': 'en-us',
 'images': [{'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_SOURCE',
   'width': 1440,
   'height': 810,
   'fallback': False},
  {'ratio': '16_9',
   'url': 'https://s1.ticketm.net/dam/c/677/c58dd0af-cda8-49b2-bdaf-068540c3a677_106541_TABLET_LANDSCAPE_LARGE_16_9.jpg',
   'width': 2048,
   'height': 1152,
   'fallback': True},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_RETINA_PORTRAIT_3_2.jpg',
   'width': 640,
   'height': 427,
   'fallback': False},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4ddd-9acd-82235bf7eee4_ARTIST_PAGE_3_2.jpg',
   'width': 305,
   'height': 203,
   'fallback': False},
  {'ratio': '3_2',
   'url': 'https://s1.ticketm.net/dam/a/ee4/8664d563-18d3-4